# Experimental new pipeline

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import rpy2.robjects as robjects
import multiprocessing
import re
import os

from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline
from pimp.settings import getString

## Load analysis and project

In [5]:
analysis = Analysis.objects.get_or_create(id=9)[0]

DEBUG 2016-09-09 11:26:59,659 utils.py:91 | (0.000) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2016-09-09 11:26:59,674 utils.py:91 | (0.014) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 9 LIMIT 21; args=(9,)


In [6]:
project = Project.objects.get_or_create(id=9)[0]

DEBUG 2016-09-09 11:27:00,257 utils.py:91 | (0.031) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 9 LIMIT 21; args=(9,)


In [7]:
pipeline = Rpy2Pipeline(analysis, project, False)

******************************************
Setup rpy2 connection
******************************************
[1] "Initialising PuMP logging"
[1] "PuMP log level set to DEBUG"
2016-09-09 11:29:46 ERROR:pump.misc.initialisePuMPLogging:Logging at ERROR level
2016-09-09 11:29:46 WARNING:pump.misc.initialisePuMPLogging:Logging at WARNING level
2016-09-09 11:29:46 INFO:pump.misc.initialisePuMPLogging:Logging at INFO level
2016-09-09 11:29:46 DEBUG:pump.misc.initialisePuMPLogging:Logging at DEBUG level
2016-09-09 11:29:47 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-09 11:29:47 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx512m
2016-09-09 11:29:47 INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-09 11:29:47 INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/

In [8]:
pipeline.setup()

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-09 11:29:55,976 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-09 11:29:56,012 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)



Setup called
2016-09-09 11:29:55 INFO:Pimp.getPimpWd:Data dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects
2016-09-09 11:29:55 INFO:Pimp.getPimpWd:Project dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9
2016-09-09 11:29:55 INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 9
2016-09-09 11:29:55 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-09 11:29:55 9:INFO:Pimp.validateInput:Input validated correctly!
2016-09-09 11:29:55 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 9
2016-09-09 11:29:55 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-09 11:29:56 9:INFO:Pimp.getAnalysisParams:Setting params
2016-09-09 11:29:56 9:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2016-09-09 11:29:56 9:DEBUG:Pimp.getAnalysisParams:Name: rsd Value: 0.5
2016-09-09 11:29:56 9:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2016-09-09 11:29:56 9:DEBUG:Pimp.getAnalysisParams:Name: p

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_9

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-09 11:29:56,261 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_9

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2016-09-09 11:29:56 9:INFO:Pimp.createAnalysisDir:Analysis dir created at analysis_9


## Peak detection

In [9]:
def get_value(r_vect, key):
    return r_vect[r_vect.names.index(key)]

In [111]:
def get_formatted_value(r_vect, key, analysis_id=None, subst=None):

    str_vect = get_value(r_vect, key)
    my_str = str_vect[0]
    
    # replace 'analysis' with 'analysis_%d'
    if analysis_id is not None:
        my_str = my_str.replace('analysis', 'analysis_%d' % analysis_id)
    
    # replace the 'polarity' bit
    if subst is not None:
        my_str = my_str % subst

    return my_str

In [10]:
def peak_detection(files, xcms_params, n_slaves):

    # collect the parameters
    args = {
        'files'           : files,
        'method'          : get_value(xcms_params, 'method'),
        'ppm'             : get_value(xcms_params, 'ppm'),
        'peakwidth'       : get_value(xcms_params, 'peakwidth'),
        'snthresh'        : get_value(xcms_params, 'snthresh'),
        'prefilter'       : get_value(xcms_params, 'prefilter'),
        'integrate'       : get_value(xcms_params, 'integrate'),
        'mzdiff'          : get_value(xcms_params, 'mzdiff'),
        'verbose.columns' : get_value(xcms_params, 'verbose.columns'),
        'fitgauss'        : get_value(xcms_params, 'fitgauss'),
        'nSlaves'         : n_slaves
    }

    # call centwave
    xcms_set = robjects.r['xcmsSet']    
    xset = xcms_set(**args)
    return xset

In [11]:
positive_files = get_value(pipeline.metadata.r_files, 'positive')
xcms_params = get_value(pipeline.pimp_params, 'xcms.params')
n_slaves = multiprocessing.cpu_count()-1

print positive_files, xcms_params, n_slaves

 [1] "samples/POS/Beer_4_full1.mzXML" "samples/POS/Beer_4_full2.mzXML"
 [3] "samples/POS/Beer_4_full3.mzXML" "samples/POS/Beer_1_full1.mzXML"
 [5] "samples/POS/Beer_1_full2.mzXML" "samples/POS/Beer_1_full3.mzXML"
 [7] "samples/POS/Beer_2_full1.mzXML" "samples/POS/Beer_2_full2.mzXML"
 [9] "samples/POS/Beer_2_full3.mzXML" "samples/POS/Beer_3_full1.mzXML"
[11] "samples/POS/Beer_3_full2.mzXML" "samples/POS/Beer_3_full3.mzXML"
 $method
[1] "centWave"

$ppm
[1] 3

$peakwidth
[1]   5 100

$snthresh
[1] 2

$prefilter
[1]   2 500

$integrate
[1] 1

$mzdiff
[1] 0.001

$verbose.columns
[1] TRUE

$fitgauss
[1] FALSE

 7


In [12]:
xset_pos = peak_detection(positive_files, xcms_params, n_slaves)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-09 11:30:19,595 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:BiocGenerics’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, parApply, parCapply,
    parLapply, parRapply, parSapply


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-09 11:30:19,597 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The f

Starting snow cluster with 7 local sockets.
Detecting features in file # 1 : Beer_4_full1.mzXML 
Detecting features in file # 2 : Beer_4_full2.mzXML 
Detecting features in file # 3 : Beer_4_full3.mzXML 
Detecting features in file # 4 : Beer_1_full1.mzXML 
Detecting features in file # 5 : Beer_1_full2.mzXML 
Detecting features in file # 6 : Beer_1_full3.mzXML 
Detecting features in file # 7 : Beer_2_full1.mzXML 
Detecting features in file # 8 : Beer_2_full2.mzXML 
Detecting features in file # 9 : Beer_2_full3.mzXML 
Detecting features in file # 10 : Beer_3_full1.mzXML 
Detecting features in file # 11 : Beer_3_full2.mzXML 
Detecting features in file # 12 : Beer_3_full3.mzXML 


## Peak alignment

In [86]:
mzmatch_params = get_value(pipeline.pimp_params, 'mzmatch.params')
peakml_params = get_value(pipeline.pimp_params, 'peakml.params')
mzmatch_outputs = get_value(pipeline.pimp_params, 'mzmatch.outputs')
method = get_formatted_value(mzmatch_params, 'rt.alignment')
print method

obiwarp


In [14]:
print mzmatch_params

$rsd
[1] 0.5

$iqr
[1] 0.5

$rt.alignment
[1] "obiwarp"

$noise
[1] 0.8

$ppm
[1] 3

$order
[1] 3

$maxrt
[1] 120

$rtwindow
[1] 30

$id.rtwindow
[1] 0.05

$combination
[1] "set"

$mindetections
[1] 3

$minintensity
[1] 5000

$adducts.positive
[1] "M+H,M+ACN+Na,M+Na,M+K,M+ACN+H"

$adducts.negative
[1] "M-H,M-ACN+Na,M-Na,M-K,M-ACN+H"




In [108]:
print mzmatch_outputs

$alignment.folder
[1] "analysis/%s/combined_rt.alignment"

$combined.folder
[1] "analysis/%s/combined"

$combined.rsd.filtered.folder
[1] "analysis/%s/combined_RSD_filtered"

$combined.rsd.rejected.folder
[1] "analysis/%s/combined_RSD_rejected"

$final.combined.peakml.file
[1] "analysis/%s/final_combined.peakml"

$final.combined.noise.filtered.file
[1] "analysis/%s/final_combined_nf.peakml"

$final.combined.simple.filtered.file
[1] "analysis/%s/final.combined_sf.peakml"

$final.combined.gapfilled.file
[1] "analysis/%s/final_combined_gapfilled.peakml"

$final.combined.related.file
[1] "analysis/%s/final_combined_related.peakml"

$final.combined.basepeaks.file
[1] "analysis/%s/final_combined_basepeaks.peakml"

$final.combined.related.id.file
[1] "analysis/%s/final_combined_related_identified.peakml"

$final.combined.related.stds.id.file
[1] "analysis/%s/final_combined_related_stds_identified.peakml"

$final.combined.basepeaks.id.file
[1] "analysis/%s/final_combined_basepeaks_identified.p

In [58]:
method = get_value(mzmatch_params, 'rt.alignment')[0]

In [17]:
xset = xset_pos

In [89]:
def rt_correct(xset, method, peakml_params, mzmatch_outputs, n_slaves, verbose):
    
    if method == 'obiwarp':
        retcor = robjects.r['retcor']
        xset_aln = retcor(xset, method='obiwarp', profStep=0.01)        
    elif method == 'loess':
        retcor = robjects.r['retcor']
        xset_aln = retcor(xset, method='loess', family="symmetric")
    else:
        xset_aln = xset
        
    filepaths = robjects.r['filepaths']
    fp = filepaths(xset_aln)

    split = robjects.r['split']
    xseto = split(xset_aln, fp)
    return fp, xseto

In [59]:
fp, xseto = rt_correct(xset, method, peakml_params, mzmatch_outputs, n_slaves, True)

center sample:  Beer_1_full2 
Processing: Beer_4_full1  Beer_4_full2  Beer_4_full3  Beer_1_full1  Beer_1_full3  Beer_2_full1  Beer_2_full2  Beer_2_full3  Beer_3_full1  Beer_3_full2  Beer_3_full3  


#### get java heapsize

In [61]:
xmx = getString('PIMP_JAVA_PARAMETERS', '')
print xmx

found = re.findall(r'\d+', xmx)
heapsize = int(found[0])
print heapsize

-Xmx8192m
8192


#### initialize mzmatch

In [62]:
# I think this should be called only once??
args = {
    'memorysize' : heapsize,
    'version.1'  : False
}
mzmatch_init = robjects.r['mzmatch.init']
mzmatch_init(**args)

2016-09-09 13:29:45 9:INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-09 13:29:45 9:INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx8192m
2016-09-09 13:29:45 9:INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-09 13:29:45 9:INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/swt-4.5-cocoa-macosx-x86_64.jar


<BoolVector - Python:0x132cd7050 / R:0x7fb141e80ba8>
[       0]

In [63]:
write_peakml = robjects.r['PeakML.xcms.write.SingleInstance']

In [112]:
polarity_folder = get_formatted_value(mzmatch_outputs, 'polarity.folder', analysis_id=analysis.id, subst='pos')

In [114]:
ionisation = get_value(peakml_params, 'ionisation')
add_scans = get_value(peakml_params, 'addscans')
write_rejected = get_value(peakml_params, 'writeRejected')
apodisation_filter = get_value(peakml_params, 'ApodisationFilter')
ppm = get_value(peakml_params, 'ppm') # not set inside peakml_params ..?
if ppm == 0:
    ppm = 5

In [115]:
peakml_files = []
regex = re.compile(re.escape('mzxml'), re.IGNORECASE)
for single_fp in fp:
    replaced = regex.sub('peakml', single_fp)
    outfile = os.path.join(polarity_folder, os.path.basename(replaced))
    peakml_files.append(outfile)

In [116]:
print peakml_files

['analysis_9/pos/Beer_4_full1.peakml', 'analysis_9/pos/Beer_4_full2.peakml', 'analysis_9/pos/Beer_4_full3.peakml', 'analysis_9/pos/Beer_1_full1.peakml', 'analysis_9/pos/Beer_1_full2.peakml', 'analysis_9/pos/Beer_1_full3.peakml', 'analysis_9/pos/Beer_2_full1.peakml', 'analysis_9/pos/Beer_2_full2.peakml', 'analysis_9/pos/Beer_2_full3.peakml', 'analysis_9/pos/Beer_3_full1.peakml', 'analysis_9/pos/Beer_3_full2.peakml', 'analysis_9/pos/Beer_3_full3.peakml']


In [120]:
directory = os.path.dirname(peakml_files[i])
if not os.path.exists(directory):
    os.makedirs(directory)

In [122]:
for i in range(len(peakml_files)):
    print 'Now creating %s' % peakml_files[i]
    write_peakml(xseto[i], outputfile=peakml_files[i], ionisation=ionisation, 
                 addscans=add_scans, writeRejected=write_rejected, 
                 ApodisationFilter=apodisation_filter, ppm=ppm)

Now creating analysis_9/pos/Beer_4_full1.peakml
2016-09-09 15:13:00 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-09 15:13:00 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2016-09-09 15:13:00 9:INFO:PeakML.xcms.write.SingleInstance:- /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/samples/POS/Beer_1_full1.mzXML
2016-09-09 15:13:21 9:INFO::11539 peaks exported.
Now creating analysis_9/pos/Beer_4_full2.peakml
2016-09-09 15:13:22 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-09 15:13:22 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2016-09-09 15:13:22 9:INFO:PeakML.xcms.write.SingleInstance:- /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/samples/POS/Beer_1_full2.mzXML
2016-09-09 15:13:46 9:INFO::11471 peaks exported.
Now creating analysis_9/pos/Beer_4_full3.peakml
2016-09-09 15:13:47 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-09 15:13:47 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data